In [1]:
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [2]:
train_df = pd.read_excel("data/train.xlsx")
test1_df = pd.read_excel("data/testA.xlsx")
test2_df = pd.read_excel("data/testB.xlsx")

In [3]:
train_y = train_df[train_df.columns[-1]]
train_y.shape

(800,)

In [4]:
new_column = test1_df.columns
train_df = train_df[new_column]
train_id = train_df['ID']
test1_id = test1_df['ID']
test2_id = test2_df['ID']
train_df.drop(['ID'], axis=1, inplace=True)
test1_df.drop(['ID'], axis=1, inplace=True)
test2_df.drop(['ID'], axis=1, inplace=True)

In [5]:
train_df = train_df[train_y>1.8]
train_df.shape

(799, 5952)

In [6]:
train_y = train_y[train_y>1.8]
train_y.shape

(799,)

In [7]:
# ints or strings
ints = []
year = []
names = train_df.columns
dts = train_df.dtypes
for i in range(len(names)):
    if 'int' in str(dts[i]) or 'object' in str(dts[i]):
        ints.append(names[i])
print(len(ints))

for n in ints:
    if str(train_df[n][0])[:4] == '2017' and str(test1_df[n][0])[:4] == '2017':
        year.append(n)
print(len(year))

770
45


In [15]:
names = train_df.columns
dts = train_df.dtypes
drop_names = []
ratio = 0.1
for i in range(len(names)):
    n = names[i]
    tp = dts[i]
    trd = train_df[n]
    ted = test1_df[n]
    if 'object' == str(tp):
        continue
    else:
        misstr = np.sum(trd.isnull())/799
        misste = np.sum(ted.isnull())/300
        if np.abs(misstr-misste) > ratio:
            drop_names.append(n)
        else:
            if 'float' in str(tp):
                trrd = trd[trd.notnull()]
                terd = ted[ted.notnull()]
                m1 = np.mean(trrd)
                s1 = np.std(trrd)
                m2 = np.mean(terd)
                s2 = np.std(terd)
                if np.abs(m1-m2)>ratio*np.abs(s1) and np.abs(s1-s2)>ratio*np.abs(s1):
                    drop_names.append(n)
print(len(drop_names))

3355


In [9]:
df_X = pd.concat([train_df, test1_df, test2_df], axis=0)

In [16]:
df_X.drop(drop_names, axis=1, inplace=True)
df_X.shape

(1511, 2597)

In [10]:
dfx_year = df_X[year]

In [11]:
for n in dfx_year.columns:
    c = dfx_year[n]
    r = c[c.notnull()]
    if len(r) < len(c):
        print(len(c) - len(r))
        dfx_year[n][c.isnull()] = np.mean(r)
        print(n)

124
330X640
124

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/magnusterra/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/magnusterra/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)



330X641
124
330X1165
124
330X1168
124
330X1169
1
520X148


In [12]:
remains1 = []
remains2 = []
tobedrop = []

for n in ints:
    lu1 = len(train_df[n].unique())
    lu2 = len(df_X[n].unique())
    if lu1 < 10 and lu1 > 1:
        remains1.append(n)
    if lu2 < 10 and lu2 > 1:
        remains2.append(n)
    if lu1 != lu2:
        tobedrop.append(n)
    
print(len(remains1), len(remains2), len(tobedrop), len(year))

175 281 407 45


In [13]:
toberemain = list(set(remains1) & set(remains2) - set(tobedrop))
print(len(toberemain))

84


In [14]:
def dict2list(dic:dict):
    ''' 将字典转化为列表 '''
    keys = dic.keys()
    vals = dic.values()
    lst = [(key, val) for key, val in zip(keys, vals)]
    return lst

def LastRemainJudge(column):
    unique = set(column)
    uf = {}
    for u in unique:
        uf[u] = 0
    for c in column:
        uf[c] += 1
    sl = sorted(dict2list(uf), key = lambda x:x[1], reverse=True)
    x = sl[0][1]/len(column)
    sl = np.array(sl)
    if x < 0.8:
        #print(sl[:, 1]/len(column))
        return True
    else:
        return False

In [17]:
lm1 = []
lm2 = []
for n in toberemain:
    if LastRemainJudge(train_df[n]):
        lm1.append(n)
    if LastRemainJudge(df_X[n]):
        lm2.append(n)
print(len(lm1), len(lm2), len(set(lm1)&set(lm2)))

53 52 52


In [18]:
names = df_X.columns
dts = df_X.dtypes
dfx_float_n = []
dfx_onehot_n = list(set(lm1) & set(lm2))
print(len(dfx_onehot_n))
for i in range(len(names)):
    n = names[i]
    d = dts[i]
    if 'float' in str(d):
        dfx_float_n.append(n)
print(len(dfx_float_n))
dfx_float = df_X[dfx_float_n]
dfx_onehot = df_X[dfx_onehot_n]
print(dfx_float.shape, dfx_onehot.shape)

52
2068
(1511, 2068) (1511, 52)


In [19]:
drop_names = []
names = dfx_float.columns
for n in names:
    x = np.sum(dfx_float[n].isnull())
    if x > 700:
        drop_names.append(n)
print(len(drop_names))

75


In [20]:
dfx_float.drop(drop_names, axis=1, inplace=True)

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
means = []
stds = []
drop_names = []
names = dfx_float.columns
for n in names:
    c = dfx_float[n]
    r = c[c.notnull()]
    tmpm = np.mean(r)
    tmps = np.std(r)
    means.append(tmpm)
    stds.append(tmps)
    if tmps == 0:
        drop_names.append(n)
print(len(set(means)), len(set(stds)))
print(len(drop_names))

1126 1068
271


In [22]:
dfx_float.drop(drop_names, axis=1, inplace=True)

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
# not move sames first, fill nan
names = dfx_float.columns
added = set()
sames = []
for i in range(len(names)):
    tmpsl = []
    tmpsl.append(i)
    for j in range(i+1, len(names)):
        if means[i] == means[j] and stds[i] == stds[j]:
            if i not in added:
                added.add(i)
            if j not in added:
                added.add(j)
                tmpsl.append(j)
    if len(tmpsl) > 1:
        sames.append(tmpsl)
print(len(sames))

162


In [24]:
names = dfx_float.columns
drop_names = []
for i in sames:
    for j in i[1:]:
        drop_names.append(names[j])
print(len(drop_names))

627


In [25]:
dfx_float.drop(drop_names, axis=1, inplace=True)

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
print(dfx_float.shape)

(1511, 1095)


In [27]:
import copy
trainx_float = copy.copy(dfx_float)
#trainx_float.drop(drop_names, axis=1, inplace=True)
print(trainx_float.shape)

(1511, 1095)


In [28]:
names = trainx_float.columns
for n in names:
    c = trainx_float[n]
    r = c[c.notnull()]
    tmpm = np.mean(r)
    tmps = np.std(r)
    assert tmps>0
    trainx_float[n][c.isnull()] = tmpm
print(np.sum(trainx_float.isnull().values))

0


In [29]:
# it shall be that there is only thre lines with nan in log1p
mid = trainx_float-np.min(trainx_float)+1
trainx_log1p = np.log1p(mid)
del mid
print(np.sum(trainx_log1p.isnull().values))

0


In [30]:
for n in names:
    m1 = np.mean(trainx_float[n])
    s1 = np.std(trainx_float[n])
    assert s1 > 0
    trainx_float[n] = (trainx_float[n]-m1)/s1
    m2 = np.mean(trainx_log1p[n])
    s2 = np.std(trainx_log1p[n])
    assert s2 > 0
    trainx_log1p[n] = (trainx_log1p[n]-m2)/s2

In [31]:
ohnames = dfx_onehot.columns
for i in range(len(ohnames)):
    n = ohnames[i]
    tmp = pd.get_dummies(dfx_onehot[n], prefix='onehot_'+str(i))
    dfx_onehot = pd.concat([dfx_onehot, tmp], axis=1)

In [32]:
dfx_onehot.drop(ohnames, axis=1, inplace=True)
print(dfx_onehot.shape)

(1511, 200)


In [33]:
dfx_float = trainx_float
dfx_log1p = trainx_log1p

In [34]:
year_n = dfx_year.columns
standardlen = len(str(dfx_year[year_n[0]][0]))
for i in range(1, len(year_n)):
    l = len(str(dfx_year[year_n[i]][0]))
    bias = standardlen - l
    b10 = 1
    if bias > 0:
        for j in range(bias):
            b10 *= 10
            #dfx_year[year_n[i]] /= 10
    elif bias < 0:
        for j in range(-bias):
            b10 /= 10
            #dfx_year[year_n[i]] *= 10
    if b10 != 1:
        dfx_year[year_n[i]] *= b10
    dfx_year[year_n[i]] = dfx_year[year_n[i]] - dfx_year[year_n[0]] 

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [35]:
dfx_time = (dfx_year - np.mean(dfx_year))/np.std(dfx_year)

In [36]:
dfx_time = dfx_time[dfx_time.columns[1:]]
dfx_time.shape

(1511, 44)

In [37]:
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance

/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [38]:
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

In [39]:
def MSE(y_raw, y_pred):
    r = np.mean(np.square(y_raw-y_pred))
    #print(r)
    return r

In [40]:
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV 

/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [41]:
from sklearn.model_selection import KFold 
kf = KFold(n_splits=5, random_state=2018)
def get_oof(rgr, xtrain, ytrain, xtest, lenot, lenoe):
    oof_train = np.zeros((lenot,))
    oof_test = np.zeros((lenoe,))
    oof_test_skf = np.empty((5, lenoe))
    for i, (train_index, test_index) in enumerate(kf.split(xtrain)):
        kf_xtrain = xtrain[train_index]
        kf_ytrain = ytrain[train_index]
        kf_xtest = xtrain[test_index]
        rgr.fit(kf_xtrain, kf_ytrain)
        oof_train[test_index] = rgr.predict(kf_xtest)
        oof_test_skf[i, : ] = rgr.predict(xtest)
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1,1), oof_test.reshape(-1,1)

In [42]:
dfx = pd.concat([dfx_log1p, dfx_onehot], axis=1)
dfx.shape

(1511, 1295)

In [43]:
ndfx = pd.concat([dfx, dfx_time], axis=1)
ndfx.shape

(1511, 1339)

In [44]:
results = []
for i in range(100):
    subxtrain, subxval, subytrain, subyval = train_test_split(ndfx[:799].values, train_y.values, test_size=0.2, random_state=i)
    
    xgb = XGBRegressor(gamma=0.0001, max_depth=4, objective='reg:linear')
    xgb.fit(subxtrain, subytrain)
    test1_pred = xgb.predict(subxval)
    t = MSE(test1_pred, subyval)
    results.append(t)
print(np.mean(results), np.std(results), np.max(results), np.min(results))

0.0157311561903 0.00179116319393 0.0216595285579 0.0122588616686


In [46]:
xgb = XGBRegressor(gamma=0.0001, max_depth=4, objective='reg:linear')
xgb.fit(ndfx[:799].values, train_y.values)
test1_pred = xgb.predict(ndfx[:799].values)
print(MSE(test1_pred, train_y.values))
test1_pred = xgb.predict(ndfx[799:1099].values)
test1_pred = test1_pred.reshape(300)
save = pd.DataFrame({'ID':test1_id, 'value':test1_pred})
save.to_csv('answer/Synchronous_std_logohtime_xgbparam_20180120.csv', index=False, header=False)

0.00149390462231
